In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
train = pd.read_csv('p3_train.csv')
test = pd.read_csv('p3_test.csv')

In [15]:
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'], test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, Attention
from tensorflow.keras.callbacks import EarlyStopping

In [17]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [18]:
X_train_tagged_texts = []
for text in X_train:
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    X_train_tagged_texts.append(tags)
X_test_tagged_texts = []
for text in X_test:
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    X_test_tagged_texts.append(tags)
# Map POS tags to integers using NLTK's tagset
tagset = {"CC": 1, "CD": 2, "DT": 3, "EX": 4, "FW": 5, "IN": 6, "JJ": 7, "JJR": 8, "JJS": 9, "LS": 10, "MD": 11,
          "NN": 12, "NNS": 13, "NNP": 14, "NNPS": 15, "PDT": 16, "POS": 17, "PRP": 18, "PRP$": 19, "RB": 20,
          "RBR": 21, "RBS": 22, "RP": 23, "SYM": 24, "TO": 25, "UH": 26, "VB": 27, "VBD": 28, "VBG": 29,
          "VBN": 30, "VBP": 31, "VBZ": 32, "WDT": 33, "WP": 34, "WP$": 35, "WRB": 36}
X_train_tagged = []
for tagged_text in X_train_tagged_texts:
    tagged_text_int = []
    for word, tag in tagged_text:
        if tag in tagset:
            tagged_text_int.append(tagset[tag])
    X_train_tagged.append(tagged_text_int)

X_test_tagged = []
for tagged_text in X_test_tagged_texts:
    tagged_text_int = []
    for word, tag in tagged_text:
        if tag in tagset:
            tagged_text_int.append(tagset[tag])
    X_test_tagged.append(tagged_text_int)


max_length = max([len(seq) for seq in X_train_tagged])
max_length = max([len(seq) for seq in X_test_tagged])
# Pad sequences to a fixed length
# max_length = 12270
X_train_padded = pad_sequences(X_train_tagged, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_tagged, maxlen=max_length, padding='post')

In [46]:

# LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tagset)+1, output_dim=300, input_length=max_length))
model.add(Conv1D(64, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(32, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=3))
model.add(Dense(32, activation='tanh'))
model.add(GlobalMaxPooling1D())
model.add(Dense(11, activation='softmax'))  # 2 classes: generated by same method or not
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)
# train the model
model.fit(X_train_padded, y_train, epochs=15, batch_size=32)


Epoch 1/15
238/238 [==============================] - 38s 129ms/step - loss: 1.7176 - accuracy: 0.3674
Epoch 2/15
238/238 [==============================] - 30s 126ms/step - loss: 1.2374 - accuracy: 0.5358
Epoch 3/15
238/238 [==============================] - 30s 125ms/step - loss: 1.1327 - accuracy: 0.5669
Epoch 4/15
238/238 [==============================] - 30s 126ms/step - loss: 1.0520 - accuracy: 0.5924
Epoch 5/15
238/238 [==============================] - 30s 126ms/step - loss: 0.9933 - accuracy: 0.6090
Epoch 6/15
238/238 [==============================] - 30s 126ms/step - loss: 0.9490 - accuracy: 0.6303
Epoch 7/15
238/238 [==============================] - 30s 126ms/step - loss: 0.9330 - accuracy: 0.6328
Epoch 8/15
238/238 [==============================] - 30s 126ms/step - loss: 0.8935 - accuracy: 0.6509
Epoch 9/15
238/238 [==============================] - 30s 125ms/step - loss: 0.8550 - accuracy: 0.6672
Epoch 10/15
238/238 [==============================] - 30s 125ms/step - l

In [47]:
# tokenizer.fit_on_texts(X_test)
loss, acc = model.evaluate(X_test_padded, y_test)

60/60 [==============================] - 4s 49ms/step - loss: 0.9413 - accuracy: 0.6481


In [48]:
preds = model.predict(X_test_padded)

60/60 [==============================] - 4s 48ms/step


In [49]:
preds[0]

array([3.4885248e-04, 9.9589074e-01, 1.2182490e-03, 4.3332661e-04,
       9.9432045e-06, 5.4924272e-04, 1.8623366e-04, 7.4970976e-06,
       1.2664089e-03, 5.3656426e-05, 3.5821951e-05], dtype=float32)

In [50]:
preds_new = np.argmax(preds, axis = 1)

In [51]:
preds_new = np.array(preds_new)

In [52]:
preds_new[0]

1

In [53]:
from sklearn.metrics import f1_score,precision_score, recall_score, accuracy_score
f1 = f1_score(y_test, preds_new, average='macro')
print("F1 score = ",f1)
precision = precision_score(y_test, preds_new, average='macro')
recall = recall_score(y_test, preds_new, average='macro')
print("Precision = ", precision)
print("Recall = ", recall)
print("Accuracy = ", accuracy_score(y_test, preds_new))

F1 score =  0.6493691022822049
Precision =  0.6675646685491493
Recall =  0.6488862444507566
Accuracy =  0.6480505795574288


In [54]:
from sklearn.metrics import classification_report, roc_curve, auc, f1_score, accuracy_score, confusion_matrix
matrix = confusion_matrix(y_test, preds_new, labels = [0, 1, 2])
mat = matrix.diagonal()/matrix.sum(axis=1)
print(classification_report(y_test, preds_new, labels = [0, 1, 2],digits=4))
print('confusion matrix: ', mat)

              precision    recall  f1-score   support

           0     0.8077    0.7590    0.7826       166
           1     0.9294    0.8827    0.9054       179
           2     0.5732    0.5434    0.5579       173

   micro avg     0.7714    0.7297    0.7500       518
   macro avg     0.7701    0.7284    0.7486       518
weighted avg     0.7714    0.7297    0.7500       518

confusion matrix:  [0.99212598 0.96932515 0.97916667]
